<a href="https://colab.research.google.com/github/Kennethdotse/ayaresa_ai/blob/main/ayaresa_ai_groq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade --quiet langchain chromadb
!pip install -U --quiet langchain-community
!pip install -U --quiet unstructured
!pip install --upgrade --quiet datasets
!pip install --upgrade --quiet accelerate bitsandbytes transformers
!pip install gradio --q
!pip install -q groq transformers sentence-transformers pypdf python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.7/20.7 MB 126.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 126.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 11.1 MB

In [ ]:
import os
from pathlib import Path
from typing import Optional, List

import pandas as pd
from groq import Groq
from PIL import Image

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_community.document_loaders.dataframe import DataFrameLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.documents import Document as LangchainDocument
from datasets import load_dataset
from docx import Document

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

files = '/content/drive/MyDrive/medical/hb_db'
import os
for file in os.listdir(files):
  print(file)

Final_Dataset.csv
west-african-herbal-pharmacopoeiaok.pdf
2005-093.pdf
dr-sebis-bible-of-natural-remedies-the-complete-collection-of-dr-sebis-alkaline-herbs-anti-inflammatory-diet-and-holistic-treatments-to-naturally-transform-your-body_compress.pdf
African-American Slave Medicine Herbal and non-Herbal Treatments.pdf
Healing_Foods_avasshop.pdf
B-001-001-223.pdf
african-holistic-health-pdf-free.pdf
The-Lost-Book-of-Herbal-Remedies.pdf


In [ ]:
# Configuration
MODEL_NAME = "llama-3.1-8b-instant"  # chosen model
LOCAL_DOCS_PATH = Path(files)
CHROMA_PERSIST_DIR = "/content/chroma_db"
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")  # set this in the environment or below

RAG_K = 3
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200



print("LOCAL_DOCS_PATH:", LOCAL_DOCS_PATH)
print("CHROMA_PERSIST_DIR:", CHROMA_PERSIST_DIR)
print("MODEL_NAME:", MODEL_NAME)

LOCAL_DOCS_PATH: /content/drive/MyDrive/medical/hb_db
CHROMA_PERSIST_DIR: /content/chroma_db
MODEL_NAME: llama-3.1-8b-instant


In [ ]:
def load_local_documents(local_path: Path) -> List[LangchainDocument]:
    docs = []
    if not local_path.exists():
        print("Warning: local docs path not found:", local_path)
        return docs

    csv_path = local_path / "Final_Dataset.csv"
    if csv_path.exists():
        try:
            csv_loader = CSVLoader(str(csv_path))
            docs += csv_loader.load()
            print(f"Loaded {len(docs)} docs from CSV")
        except Exception as e:
            print("CSV loader error:", e)

    for txt_file in local_path.glob("*.txt"):
        try:
            txt_loader = TextLoader(str(txt_file))
            docs += txt_loader.load()
            print(f"Loaded {txt_file.name} (txt)")
        except Exception as e:
            print("TXT loader error:", e)

    for pdf_file in local_path.glob("*.pdf"):
        try:
            pdf_loader = PyPDFLoader(str(pdf_file))
            loaded = pdf_loader.load()
            docs += loaded
            print(f"Loaded {pdf_file.name} with {len(loaded)} pages")
        except Exception as e:
            print("PDF loader error for", pdf_file, e)

    return docs

In [ ]:
def load_hf_medical_dataset() -> List[LangchainDocument]:
    docs = []
    try:
        ds = load_dataset("knowrohit07/know_medical_dialogue_v2")
        if "train" in ds:
            df = pd.DataFrame(ds["train"])
            if "instruction" in df.columns and "output" in df.columns:
                df["full_dialogue"] = df["instruction"].astype(str) + "\n\n" + df["output"].astype(str)
                loader = DataFrameLoader(df, page_content_column="full_dialogue")
                docs += loader.load()
                print("Loaded dataset rows into docs:", len(docs))
    except Exception as e:
        print("Warning: couldn't load HF dataset:", e)
    return docs

local_docs = load_local_documents(LOCAL_DOCS_PATH)
hf_docs = load_hf_medical_dataset()
all_docs = local_docs + hf_docs
print("Total documents loaded:", len(all_docs))

Loaded 49 docs from CSV
Loaded west-african-herbal-pharmacopoeiaok.pdf with 260 pages
Loaded 2005-093.pdf with 269 pages
Loaded dr-sebis-bible-of-natural-remedies-the-complete-collection-of-dr-sebis-alkaline-herbs-anti-inflammatory-diet-and-holistic-treatments-to-naturally-transform-your-body_compress.pdf with 218 pages
Loaded African-American Slave Medicine Herbal and non-Herbal Treatments.pdf with 217 pages
Loaded Healing_Foods_avasshop.pdf with 354 pages
Loaded B-001-001-223.pdf with 412 pages
Loaded african-holistic-health-pdf-free.pdf with 597 pages
Loaded The-Lost-Book-of-Herbal-Remedies.pdf with 306 pages


README.md: 0.00B [00:00, ?B/s]

know_med_v4.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/6307 [00:00<?, ? examples/s]

Loaded dataset rows into docs: 6307
Total documents loaded: 8989


In [ ]:
splitter = CharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
chunks = splitter.split_documents(all_docs)
print("Number of chunks:", len(chunks))

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# If persisted DB exists, load it; otherwise create anew
from pathlib import Path
if Path(CHROMA_PERSIST_DIR).exists():
    print("Loading persisted Chroma DB...")
    vectordb = Chroma(persist_directory=CHROMA_PERSIST_DIR, embedding_function=embeddings)
else:
    print("Creating Chroma DB from chunks (computing embeddings)...")
    vectordb = Chroma.from_documents(chunks, embeddings, persist_directory=CHROMA_PERSIST_DIR)
    try:
        vectordb.persist()
    except Exception:
        pass

print("Vectorstore ready.")

/tmp/ipython-input-89461947.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


Number of chunks: 9129


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Creating Chroma DB from chunks (computing embeddings)...
Vectorstore ready.


/tmp/ipython-input-89461947.py:17: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [ ]:
import re


def retrieve_topk_and_bullets(query: str, k: int = 3) -> (List[str], str):
    """Retrieve top-k chunks and return both the chunk texts and a bullet summary.

    Bullet strategy (simple and fast):
    - For each chunk, split into sentences and pick the most informative sentences (heuristic: first and longest sentence)
    - Compose bullets by extracting up to 2 sentences per chunk and short-trimming them
    """
    hits = vectordb.similarity_search(query, k=k)
    chunk_texts = [d.page_content for d in hits]

    bullets = []
    for ct in chunk_texts:
        # naive sentence split
        sents = re.split(r'(?<=[\.!?。！？])\s+', ct.strip())
        # pick up to two candidate sentences: prefer longer sentences
        sents_sorted = sorted(sents, key=lambda s: len(s), reverse=True)
        picked = []
        if len(sents_sorted) > 0:
            picked.append(sents_sorted[0])
        if len(sents_sorted) > 1:
            picked.append(sents_sorted[1])
        # trim and add bullets
        for p in picked[:2]:
            summary = p.strip()
            # short truncation
            if len(summary) > 300:
                summary = summary[:297].rsplit(' ', 1)[0] + '...'
            bullets.append("- " + summary)

    bullet_text = "\n".join(bullets)
    return chunk_texts, bullet_text

In [ ]:
def build_rag_prompt_from_bullets(bullet_text: str, query: str) -> str:
    context = bullet_text
    rag_prompt = f"""
[System]: You are a respectful and knowledgeable medical AI assistant.
Use the provided context and your knowledge to answer clearly and do not include system or context in your final response.

[Context]: {context}

[User]: {query}

[Assistant]:
"""
    return rag_prompt

In [ ]:
if GROQ_API_KEY is None:
    raise RuntimeError("Set GROQ_API_KEY in the environment before running the Groq call")

client = Groq(api_key=GROQ_API_KEY)

In [ ]:
def call_groq_chat(system_message: str, user_message: str, model_name: str = MODEL_NAME, timeout: int = 180):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ]
    resp = client.chat.completions.create(
        timeout=timeout,
        model=model_name,
        messages=messages,
    )
    try:
        return resp.choices[0].message.content
    except Exception:
        return str(resp)

In [ ]:
# Short system message (we send the short system plus RAG prompt as user, per your instruction)
system_msg = "You are a helpful medical assistant assistant."


def rag_and_call_groq(question: str, k: int = RAG_K) -> str:
    # Retrieve top-k and produce bullets
    chunk_texts, bullets = retrieve_topk_and_bullets(question, k=k)
    if len(chunk_texts) == 0:
        bullets = "No context documents found."

    rag_prompt = build_rag_prompt_from_bullets(bullets, question)

    # Call Groq
    raw = call_groq_chat(system_message=system_msg, user_message=rag_prompt, model_name=MODEL_NAME)

    if isinstance(raw, str) and raw.strip().startswith(rag_prompt.strip()):
        raw = raw[len(rag_prompt):].strip()

    for prefix in ("[System]:", "[Context]:", "[User]:", "[Assistant]:"):
        if raw.startswith(prefix):
            raw = raw[len(prefix):].strip()

    return raw

print(rag_and_call_groq("I have chills and body pains. What could this be?"))


In [ ]:
question = "I have body pains and chills. What could be a possible cause and whats a herbal way to treat it?"
answer = rag_and_call_groq(question)
print("Q:", question)
print("\nA:\n", answer)

Q: I have body pains and chills. What could be a possible cause and whats a herbal way to treat it?

A:
 I'm happy to help you explore some possible causes and herbal remedies for your symptoms. 

Considering your symptoms of body pains and chills, it's essential to consult a medical professional for an accurate diagnosis. However, looking at the provided context, we can discuss some possibilities.

The symptoms you've mentioned could be related to prostate inflammation, which often presents with fever, chills, lower abdominal or back pain, and possibly other symptoms like frequent urination. Another possible cause mentioned is pneumonia, characterized by coughing, respiratory problems, and inflammation in the lungs.

In terms of herbal remedies, one option mentioned in "The Lost Book of Herbal Remedies" is using a salve made from Lobelia inflata roots, which can treat body aches, muscle, and joint pain.

For pneumonia, while it's critical to seek medical attention immediately for trea

USER INTERFACE WITH GRADIO

In [ ]:
import os
from datetime import datetime, timedelta
import random
from functools import partial
import gradio as gr
import threading
from docx import Document

In [ ]:
def save_to_word(prompt_text: str, response_text: str, filename: str = "prompt_response.docx") -> str:
    doc = Document()
    doc.add_heading("Prompt", level=1)
    doc.add_paragraph(prompt_text)
    doc.add_heading("Response", level=1)
    doc.add_paragraph(response_text)
    doc.save(filename)
    return filename

In [ ]:
css = """
gradio-app {
    background: none !important;
}
.md .container {
    border:1px solid #ccc;
    border-radius:5px;
    min-height:300px;
    color: #666;
    display: flex;
    justify-content: center;
    align-items: center;
    text-align: center;
    font-family: monospace;
    padding: 10px;
}
#hf_token_box {
    transition: height 1s ease-out, opacity 1s ease-out;
}
#hf_token_box.abc {
    height: 0;
    opacity: 0;
    overflow: hidden;
}
#generate_button {
    transition: background-color 1s ease-out, color 1s ease-out; border-color 1s ease-out;
}
#generate_button.changed {
    background: black !important;
    border-color: black !important;
    color: white !important;
}
"""

In [ ]:
def random_prompt():
    return "What are the herbal remedies for a cough?"

def code():
  return("```holdup```")

def toggle_view(output_visible):
    return not output_visible, gr.update(visible=not output_visible), gr.update(visible=output_visible), "View Output" if not output_visible else "View Code"

def generate_and_save(prompt_text):
    # This function should call your RAG and Groq functions and then save to word
    response_text = rag_and_call_groq(prompt_text)
    return response_text, response_text # Return response_text twice, once for display and once for state


with gr.Blocks(css=css, theme="NoCrypt/miku") as demo:
    gr.Markdown("<center><h1>AYARESA AI</h1></center>")
    gr.Markdown("Enter your prompt below to interact with a medical chatbot.")
    prompt = gr.Textbox(random_prompt, lines=2, show_label=False, info="Type your prompt here.")

    with gr.Group():
        with gr.Row():
            generate_btn = gr.Button(value="Generate", elem_id="generate_button", variant="primary", size="sm")
            code_btn = gr.Button(value="View Code", elem_id="code_button", variant="secondary", size="sm")
            save_btn = gr.Button(value="Save to Word", elem_id="save_button", variant="secondary", size="sm")

    response_state = gr.State("")

    with gr.Row() as output_row:
        llama_output = gr.Markdown("<div class='container'>Llama 3-70B Instruct</div>", elem_classes=["md"], height=300)

    with gr.Row(visible=False) as code_row:
        code_display = gr.Code(language="python", label="Full Script", value=code)

    output_visible = gr.State(True)

    code_btn.click(
        fn=toggle_view,
        inputs=[output_visible],
        outputs=[output_visible, output_row, code_row, code_btn],
    )

    generate_btn.click(
        fn=generate_and_save,
        inputs=[prompt],
        outputs=[llama_output, response_state],
    )

    save_btn.click(
        fn=save_to_word,
        inputs=[prompt, response_state],
        outputs=gr.File(label="Download Word Document"),
    )

In [ ]:
 demo.launch(show_api=False)